In [1]:
# 1

# New Notebook to run the individual_flashes_plots.py routine
# that will be used to test the new LAMPFLASH files from
# 15 July 2021, which requires taking fresh RAWTAG files
# and running CalCOS on them with WAVECOR set to PERFORM
# so that they create LAMPFLASH files that can be compared
# with LAMPFLASH files that have *not* been shifted
# (which already exist).
#
# I re-downloaded the fresh files from /grp/hst/cos2/cosmo/16469/
# and put them at
# /Users/ahirschauer/Documents/Year4/04-2021/Lamp_Tabs/ahirscha58916/data_backup_22July2021,
# then took the relevant _lampflash.fits files and put them at
# /Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots,
# with relevant sub-directories denoting the G140L or G130M data.
#
# Then I copy+paste a revised "individual_flashes_plots.py" routine
# into each of the next two Cells in order to produce the plots.


In [2]:
# 2

# I'm going to copy+paste the whole routine right here as a test, with new directories:

from astropy.io import fits
import glob
import os
import numpy as np

from matplotlib import pyplot as plt

#-------------------------------------------------------------------------------

def get_shifts(shifts):

    new_shifts = []
    for shift in shifts[1:]:
        new_shifts.append(shift-shifts[0])

    #print('{:5} {:3} {:6} {:10}'.format(cenwave, fp, segment, max_shift ))
    return new_shifts

#-------------------------------------------------------------------------------

def make_figures(lamp_list):

    for lamp in lamp_list:

        outdir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G140L/output_G140L/'
        initial_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G140L/pre-shift_G140L/'
        new_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G140L/July22_post-shift_G140L/'
                
        lamp_init = os.path.join(initial_dir, lamp)
        lamp_fin = os.path.join(new_dir, lamp)

        colors = {1: 'red',
                  2: 'blue',
                  3: 'orange',
                  4: 'green'}

        for segment in ['FUVA', 'FUVB']:
            plt.figure(figsize=(10, 5))
            shifts = []
            for lamp_file in [lamp_init, lamp_fin]:

                if 'July22' in lamp_file:
                    marker = 'x'
                else:
                    marker = '.'

                with fits.open(lamp_file) as lf: # reading in the lamptab file

                    lamptab = lf[0].header['LAMPTAB']
                    if 'lref' in lamptab:
                        lamptab = lamptab.replace('lref$', '/grp/hst/cdbs/lref/')

                    print ('-----------------')
                    print (lamptab)
                    cen = lf[0].header['cenwave']
                    fp = lf[0].header['fpoffset']
                    print (cen,fp+3,segment)
                    with fits.open(lamptab) as lt:
                        wh_tab = np.where((lt[1].data['cenwave'] == cen) &
                                          (lt[1].data['fpoffset'] == fp) &
                                          (lt[1].data['segment'] == segment))

                        if len(wh_tab[0]) == 0:
                            print('no {} {} {}'.format(cen, fp, segment))
                            continue
                        fp_pix_shift = lt[1].data['FP_PIXEL_SHIFT'][wh_tab] #finding FP_PIXSHIFT for that lampflash

                    wh_seg = np.where(lf[1].data['segment'] == segment)

                    #shift_disp is the shift between tagflash wavecal and template wavecal in dispersion direction
                    #i.e. the FP_shift + any drift
                    shift_disp = lf[1].data['SHIFT_DISP'][wh_seg]
                    print ('shift_disp:')
                    print (shift_disp)

                    shift_disp = shift_disp - fp_pix_shift
                    # since comparison was made between de-drifted corrtags and lamptemplate corrected for drift,
                    # the resultant value should be the FP shifts. Subtracting FP shifts should give zero
                    print ('shift_disp - FP_pixshift:')
                    print (shift_disp)

                    new_shifts = get_shifts(shift_disp)
                    print ('shifts relative to the first one (i.e. drift)')
                    print(new_shifts)

                    xvals = np.arange(len(new_shifts)) + 2
                    plt.plot(xvals,new_shifts, marker=marker, color=colors[fp+3], markersize=15, linestyle="None",
                             label='{} FP{}'.format(lamp_file.split('/')[-2], lf[0].header['FPPOS']))

                    shifts.append(new_shifts)

            plt.title('{} FP{} {}'.format(cen, fp+3, segment))
            plt.xlabel('Exposure after 1st')
            plt.ylabel('Drift Relative to 1st Exposure')
            plt.axhline(0, ls='--', color='black')
            plt.axhline(-0.5, ls=':', color='orchid')
            plt.axhline(0.5, ls=':', color='orchid')

            #plt.legend(bbox_to_anchor=(0.2, 0.98), ncol=4)
            plt.legend(loc='upper left', ncol=4)

            shifts = np.array(shifts)
            if len(shifts) == 0:
                plt.close()
                continue
            if np.max(abs(shifts)) > 1.0:
                plt.ylim(-2.0, 2.0)
#                plt.ylim(-1*(np.max(abs(shifts)))-0.1, (np.max(abs(shifts))+0.1) )
            else:
                plt.ylim(-1.0, 1.0)

            outfile = os.path.join(outdir, '{}_{}_{}_compare.png'.format(cen, fp+3, segment))
            plt.savefig(outfile)
            print('Saved: {}'.format(outfile))
            plt.close()


if __name__ == "__main__":

    outdir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G140L/output_G140L/'
    initial_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G140L/pre-shift_G140L/'
    new_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G140L/July22_post-shift_G140L/'
 
    lamp_bases = [os.path.basename(f) for f in glob.glob(os.path.join(new_dir, '*lampflash*'))]
    make_figures(lamp_bases)


-----------------
15July2021_G140L_interim_lamp.fits
800 3 FUVA
shift_disp:
[ 0.11659164 -0.03660264 -0.0785535  -0.07130013 -0.11723222  0.01544972
  0.07559481  0.08892506]
shift_disp - FP_pixshift:
[ 0.11659164 -0.03660264 -0.0785535  -0.07130013 -0.11723222  0.01544972
  0.07559481  0.08892506]
shifts relative to the first one (i.e. drift)
[-0.15319428220391273, -0.1951451376080513, -0.18789177387952805, -0.23382385820150375, -0.1011419165879488, -0.04099682718515396, -0.027666576206684113]
-----------------
/grp/hst/cos2/LP5_ERA/files_to_use/LP3_G140L_fuv_15July2021_lamp.fits
800 3 FUVA
shift_disp:
[ 0.03268308 -0.14690617 -0.34793383 -0.19750586 -0.12715347 -0.07924046
 -0.01473273  0.01327338]
shift_disp - FP_pixshift:
[ 0.03268308 -0.14690617 -0.34793383 -0.19750586 -0.12715347 -0.07924046
 -0.01473273  0.01327338]
shifts relative to the first one (i.e. drift)
[-0.17958924919366837, -0.3806169107556343, -0.23018894344568253, -0.1598365530371666, -0.11192353814840317, -0.0474158

No handles with labels found to put in legend.


no 800 0 FUVB
-----------------
15July2021_G140L_interim_lamp.fits
800 1 FUVA
shift_disp:
[-476.78354 -477.01746 -477.34692 -477.42688 -477.56592 -477.72626
 -477.66672 -477.66388]
shift_disp - FP_pixshift:
[ 0.60236678  0.36844954  0.03898177 -0.04097428 -0.18001237 -0.34035173
 -0.28081193 -0.2779738 ]
shifts relative to the first one (i.e. drift)
[-0.233917236328125, -0.563385009765625, -0.643341064453125, -0.782379150390625, -0.942718505859375, -0.8831787109375, -0.880340576171875]
-----------------
/grp/hst/cos2/LP5_ERA/files_to_use/LP3_G140L_fuv_15July2021_lamp.fits
800 1 FUVA
shift_disp:
[-476.73013 -477.0219  -477.54572 -477.64398 -477.7987  -477.99365
 -477.91428 -477.90805]
shift_disp - FP_pixshift:
[ 0.18836254 -0.10341602 -0.62721973 -0.72548633 -0.88021045 -1.07515674
 -0.99578052 -0.98955494]
shifts relative to the first one (i.e. drift)
[-0.291778564453125, -0.815582275390625, -0.913848876953125, -1.068572998046875, -1.263519287109375, -1.18414306640625, -1.1779174804687

No handles with labels found to put in legend.


no 800 -2 FUVB
-----------------
15July2021_G140L_interim_lamp.fits
800 4 FUVA
shift_disp:
[269.8944  269.98593 269.8018  269.98767 269.7574  269.8053  269.87
 269.9493 ]
shift_disp - FP_pixshift:
[ 0.03648285  0.12800507 -0.056138    0.12974457 -0.10054107 -0.05262847
  0.01206879  0.09138398]
shifts relative to the first one (i.e. drift)
[0.091522216796875, -0.092620849609375, 0.09326171875, -0.13702392578125, -0.089111328125, -0.0244140625, 0.054901123046875]
-----------------
/grp/hst/cos2/LP5_ERA/files_to_use/LP3_G140L_fuv_15July2021_lamp.fits
800 4 FUVA
shift_disp:
[269.7773  269.8758  269.71393 269.9153  269.6125  269.68527 269.75616
 269.8699 ]
shift_disp - FP_pixshift:
[ 0.01828724  0.11676747 -0.04509777  0.15628773 -0.1465382  -0.07375377
 -0.00286144  0.11087757]
shifts relative to the first one (i.e. drift)
[0.098480224609375, -0.063385009765625, 0.13800048828125, -0.164825439453125, -0.092041015625, -0.021148681640625, 0.09259033203125]


No handles with labels found to put in legend.


Saved: /Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G140L/output_G140L/800_4_FUVA_compare.png
-----------------
15July2021_G140L_interim_lamp.fits
800 4 FUVB
no 800 1 FUVB
-----------------
/grp/hst/cos2/LP5_ERA/files_to_use/LP3_G140L_fuv_15July2021_lamp.fits
800 4 FUVB
no 800 1 FUVB
-----------------
15July2021_G140L_interim_lamp.fits
800 2 FUVA
shift_disp:
[-221.56456 -221.45482 -221.35374 -221.59808 -221.51901 -221.53752
 -221.39552 -221.4138 ]
shift_disp - FP_pixshift:
[-0.0799524   0.02978881  0.13086303 -0.11347596 -0.03440491 -0.05291383
  0.08908447  0.07080444]
shifts relative to the first one (i.e. drift)
[0.1097412109375, 0.2108154296875, -0.0335235595703125, 0.0455474853515625, 0.02703857421875, 0.169036865234375, 0.1507568359375]
-----------------
/grp/hst/cos2/LP5_ERA/files_to_use/LP3_G140L_fuv_15July2021_lamp.fits
800 2 FUVA
shift_disp:
[-221.76164 -221.63542 -221.52032 -221.8027  -221.70029 -221.72777
 -221.56686 -221.5757 ]
shift_disp - 

No handles with labels found to put in legend.


In [3]:
# 3

from astropy.io import fits
import glob
import os
import numpy as np

from matplotlib import pyplot as plt

#-------------------------------------------------------------------------------

def get_shifts(shifts):

    new_shifts = []
    for shift in shifts[1:]:
        new_shifts.append(shift-shifts[0])

    #print('{:5} {:3} {:6} {:10}'.format(cenwave, fp, segment, max_shift ))
    return new_shifts

#-------------------------------------------------------------------------------

def make_figures(lamp_list):

    for lamp in lamp_list:

        outdir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/output_G130M/'
        initial_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/pre-shift_G130M/'
        new_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/July22_post-shift_G130M/'

        lamp_init = os.path.join(initial_dir, lamp)
        lamp_fin = os.path.join(new_dir, lamp)

        colors = {1: 'red',
                  2: 'blue',
                  3: 'orange',
                  4: 'green'}

        for segment in ['FUVA', 'FUVB']:
            plt.figure(figsize=(10, 5))
            shifts = []
            for lamp_file in [lamp_init, lamp_fin]:

                if 'July22' in lamp_file:
                    marker = 'x'
                else:
                    marker = '.'

                with fits.open(lamp_file) as lf: # reading in the lamptab file

                    lamptab = lf[0].header['LAMPTAB']
                    if 'lref' in lamptab:
                        lamptab = lamptab.replace('lref$', '/grp/hst/cdbs/lref/')

                    print ('-----------------')
                    print (lamptab)
                    cen = lf[0].header['cenwave']
                    fp = lf[0].header['fpoffset']
                    print (cen,fp+3,segment)
                    with fits.open(lamptab) as lt:
                        wh_tab = np.where((lt[1].data['cenwave'] == cen) &
                                          (lt[1].data['fpoffset'] == fp) &
                                          (lt[1].data['segment'] == segment))

                        if len(wh_tab[0]) == 0:
                            print('no {} {} {}'.format(cen, fp, segment))
                            continue
                        fp_pix_shift = lt[1].data['FP_PIXEL_SHIFT'][wh_tab] #finding FP_PIXSHIFT for that lampflash

                    wh_seg = np.where(lf[1].data['segment'] == segment)

                    #shift_disp is the shift between tagflash wavecal and template wavecal in dispersion direction
                    #i.e. the FP_shift + any drift
                    shift_disp = lf[1].data['SHIFT_DISP'][wh_seg]
                    print ('shift_disp:')
                    print (shift_disp)

                    shift_disp = shift_disp - fp_pix_shift
                    # since comparison was made between de-drifted corrtags and lamptemplate corrected for drift,
                    # the resultant value should be the FP shifts. Subtracting FP shifts should give zero
                    print ('shift_disp - FP_pixshift:')
                    print (shift_disp)

                    new_shifts = get_shifts(shift_disp)
                    print ('shifts relative to the first one (i.e. drift)')
                    print(new_shifts)

                    xvals = np.arange(len(new_shifts)) + 2
                    plt.plot(xvals,new_shifts, marker=marker, color=colors[fp+3], markersize=15, linestyle="None",
                             label='{} FP{}'.format(lamp_file.split('/')[-2], lf[0].header['FPPOS']))

                    shifts.append(new_shifts)

            plt.title('{} FP{} {}'.format(cen, fp+3, segment))
            plt.xlabel('Exposure after 1st')
            plt.ylabel('Drift Relative to 1st Exposure')
            plt.axhline(0, ls='--', color='black')
            plt.axhline(-0.5, ls=':', color='orchid')
            plt.axhline(0.5, ls=':', color='orchid')

            #plt.legend(bbox_to_anchor=(0.2, 0.98), ncol=4)
            plt.legend(loc='upper left', ncol=4)

            shifts = np.array(shifts)
            if len(shifts) == 0:
                plt.close()
                continue
            if np.max(abs(shifts)) > 1.0:
                plt.ylim(-2.0, 2.0)
#                plt.ylim(-1*(np.max(abs(shifts)))-0.1, (np.max(abs(shifts))+0.1) )
            else:
                plt.ylim(-1.0, 1.0)

            outfile = os.path.join(outdir, '{}_{}_{}_compare.png'.format(cen, fp+3, segment))
            plt.savefig(outfile)
            print('Saved: {}'.format(outfile))
            plt.close()


if __name__ == "__main__":

    outdir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/output_G130M/'
    initial_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/pre-shift_G130M/'
    new_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/July22_post-shift_G130M/'

    lamp_bases = [os.path.basename(f) for f in glob.glob(os.path.join(new_dir, '*lampflash*'))]
    make_figures(lamp_bases)


-----------------
15July2021_G130M_interim_lamp.fits
1327 3 FUVA
shift_disp:
[ 0.3927489   0.03024902 -0.15596582 -0.23358068]
shift_disp - FP_pixshift:
[ 0.39274889  0.03024902 -0.15596582 -0.23358068]
shifts relative to the first one (i.e. drift)
[-0.36249987222254276, -0.5487147122621536, -0.6263295710086823]
-----------------
/grp/hst/cos2/LP5_ERA/files_to_use/LP5_G130M_fuv_15July2021_lamp.fits
1327 3 FUVA
shift_disp:
[ 0.15093443 -0.2785117  -0.6677163  -0.7413513 ]
shift_disp - FP_pixshift:
[ 0.15093443 -0.2785117  -0.66771632 -0.74135131]
shifts relative to the first one (i.e. drift)
[-0.42944613099098206, -0.8186507523059845, -0.8922857344150543]
Saved: /Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/output_G130M/1327_3_FUVA_compare.png
-----------------
15July2021_G130M_interim_lamp.fits
1327 3 FUVB
shift_disp:
[ 0.44090557  0.04249851 -0.15980552 -0.21207766]
shift_disp - FP_pixshift:
[ 0.44090557  0.04249851 -0.15980552 -0.21207766]
shifts 

shift_disp:
[360.38052 360.29968 360.2145  360.4169 ]
shift_disp - FP_pixshift:
[ 0.01851776 -0.06232331 -0.14749787  0.05489471]
shifts relative to the first one (i.e. drift)
[-0.080841064453125, -0.166015625, 0.036376953125]
Saved: /Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/output_G130M/1309_4_FUVA_compare.png
-----------------
15July2021_G130M_interim_lamp.fits
1309 4 FUVB
shift_disp:
[360.80695 360.80212 360.7208  360.7952 ]
shift_disp - FP_pixshift:
[ 0.03251116  0.02768938 -0.05363997  0.02076189]
shifts relative to the first one (i.e. drift)
[-0.00482177734375, -0.086151123046875, -0.011749267578125]
-----------------
/grp/hst/cos2/LP5_ERA/files_to_use/LP5_G130M_fuv_15July2021_lamp.fits
1309 4 FUVB
shift_disp:
[360.7244  360.7244  360.62814 360.712  ]
shift_disp - FP_pixshift:
[ 0.0163695   0.0163695  -0.07988294  0.00397937]
shifts relative to the first one (i.e. drift)
[0.0, -0.09625244140625, -0.01239013671875]
Saved: /Users/ahirschauer

shift_disp:
[-109.06681  -109.04639  -109.18995  -109.035934]
shift_disp - FP_pixshift:
[-0.00444721  0.01597668 -0.12758564  0.02642895]
shifts relative to the first one (i.e. drift)
[0.02042388916015625, -0.123138427734375, 0.03087615966796875]
Saved: /Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/output_G130M/1291_2_FUVB_compare.png
-----------------
15July2021_G130M_interim_lamp.fits
1291 4 FUVA
shift_disp:
[385.2607  385.12762 385.10376 385.08362]
shift_disp - FP_pixshift:
[ 0.11853353 -0.01455363 -0.03841838 -0.05855998]
shifts relative to the first one (i.e. drift)
[-0.133087158203125, -0.156951904296875, -0.177093505859375]
-----------------
/grp/hst/cos2/LP5_ERA/files_to_use/LP5_G130M_fuv_15July2021_lamp.fits
1291 4 FUVA
shift_disp:
[385.08997 384.9461  384.91266 384.90054]
shift_disp - FP_pixshift:
[ 0.03678725 -0.10707261 -0.14051988 -0.15263536]
shifts relative to the first one (i.e. drift)
[-0.14385986328125, -0.17730712890625, -0.189422

Saved: /Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/output_G130M/1318_4_FUVA_compare.png
-----------------
15July2021_G130M_interim_lamp.fits
1318 4 FUVB
shift_disp:
[274.47586 274.3143  274.23923 274.28317]
shift_disp - FP_pixshift:
[ 0.1674455   0.00588544 -0.0691878  -0.02524249]
shifts relative to the first one (i.e. drift)
[-0.16156005859375, -0.23663330078125, -0.19268798828125]
-----------------
/grp/hst/cos2/LP5_ERA/files_to_use/LP5_G130M_fuv_15July2021_lamp.fits
1318 4 FUVB
shift_disp:
[274.29718 274.1191  274.04163 274.10202]
shift_disp - FP_pixshift:
[ 0.04276039 -0.13530968 -0.21279381 -0.15239952]
shifts relative to the first one (i.e. drift)
[-0.178070068359375, -0.25555419921875, -0.195159912109375]
Saved: /Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/G130M/output_G130M/1318_4_FUVB_compare.png
-----------------
15July2021_G130M_interim_lamp.fits
1291 3 FUVA
shift_disp:
[ 0.21883401 -0.02955832 -0.134982